# Importing Data

In [1]:
# import libraries
from bs4 import BeautifulSoup
from urllib import request
import time
import random
import numpy as np
import pandas as pd
import re

## CPU Hyperlinks Table

In [2]:
## We create a csv table to store all the hyperlinks of CPU
filename = "cpu_hyperlinks.csv"
f = open(filename, "w", encoding='utf-8')
headers = "product_detail,hyperlink\n"
f.write(headers)

26

## Web Scraping all Hyperlinks

In [3]:
# Scraping 100 pages from newegg

for i in range(1, 14):
    my_url = "https://www.newegg.com/p/pl?N=100007671%20601306860&Page={}".format(
        i
    )
    uClient = request.urlopen(my_url)
    page_html = uClient.read()
    uClient.close()
    page_soup = BeautifulSoup(page_html, "html.parser")
    containers = page_soup.findAll("div", {"class": "item-container"})
    
    for container in containers:
        ## Find hyperlink that directs to the webpage of that particular product
        hyperlink = list(container.children)[3]['href']
        
        product_detail = list(container.children)[3].img["alt"].replace(",", "  ")

        
        f.write(product_detail+","+hyperlink+"\n")
        
        time.sleep(random.random()+0.5)
    time.sleep(10+random.random())
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13


In [5]:
f.close()

In [4]:
hyperlinks=pd.read_csv("cpu_hyperlinks.csv")

In [37]:
hyperlinks.head()

,product_detail,hyperlink
0,Intel Core i9-9900K Coffee Lake 8-Core 16-Th...,https://www.newegg.com/core-i9-9th-gen-intel-c...
1,Intel Core i7-9700K Coffee Lake 8-Core 3.6 GHz...,https://www.newegg.com/core-i7-9th-gen-intel-c...
2,Intel Core i7-9700KF Coffee Lake 8-Core 3.6 GH...,https://www.newegg.com/core-i7-9th-gen-intel-c...
3,Intel Core i5-9600KF Coffee Lake 6-Core 3.7 GH...,https://www.newegg.com/core-i5-9th-gen-intel-c...
4,Intel Core i3-8100 Coffee Lake Quad-Core 3.6 G...,https://www.newegg.com/core-i3-8th-gen-intel-c...


In [14]:
hyperlinks['hyperlink'][1]

'https://www.newegg.com/core-i7-9th-gen-intel-core-i7-9700k/p/N82E16819117958'

In [13]:
hyperlinks.rename(columns={"hyperlink ": "hyperlink"},inplace=True)

## Here we start to create a table of specs

In [65]:
## all cpu specs will be stored in the list total_specs
## then total_specs will be stored in a csv table
total_specs=[]
i=0

In [ ]:
for url in hyperlinks['hyperlink']:
    uClient = request.urlopen(url)
    page_html = uClient.read()
    uClient.close()
    page_soup = BeautifulSoup(page_html, "lxml")  
    
    Specs={}
    specs = page_soup.find("div", {"id": "Specs"})
    fieldsets=specs.findAll("fieldset")
    for fieldset in fieldsets:
        dls=fieldset.findAll("dl")
        for dl in dls:
            Specs[dl.find("dt").text]=dl.find("dd").text

    total_specs.append(Specs)
    i+=1
    print(i)
    
    time.sleep(10+random.random())

In [91]:
len(total_specs)

448

In [ ]:
for cpu in total_specs:
    print(len(cpu))

In [72]:
## find the maximal number of header titles for the csv file
headers=total_specs[0]
for i in total_specs[1:]:
    for j in i.keys():
        if j not in headers:
            headers[j]=None       

In [78]:
headerstr=''
for i in list(headers.keys()):
    headerstr=headerstr+i+','
headerstr=headerstr+'\n'

In [79]:
headerstr

'Brand,Processors Type,Series,Name,Model,CPU Socket Type,Core Name,# of Cores,# of Threads,Operating Frequency,Max Turbo Frequency,L3 Cache,Manufacturing Tech,64-Bit Support,Hyper-Threading Support,Memory Types,Memory Channel,Virtualization Technology Support,Integrated Graphics,Graphics Base Frequency,Graphics Max Dynamic Frequency,PCI Express Revision,Max Number of PCI Express Lanes,Thermal Design Power,Cooling Device,Date First Available,L2 Cache,Cooling Devices,Multimedia Instruction,Voltage,Options,DMI,Integrated Memory Controller Speed,QPI,FSB,L1 Cache,Hyper-Transport Support,\n'

In [113]:
f = open(filename, "w")
f.truncate()
f.close()

In [114]:
## We create a csv table to store all the hyperlinks of CPU
filename = "cpu_specs.csv"
f = open(filename, "w", encoding='utf-8')
f.write(headerstr)

589

In [115]:
headers_list=list(headers.keys())
for cpu in total_specs:
    cpu_str=''
    for col in headers_list:
        cpu_str=cpu_str+str(cpu.get(col,'None')).replace(',',' ')+','
    cpu_str=cpu_str[:-1]
    cpu_str=cpu_str+'\n'
    f.write(cpu_str) 

In [116]:
f.close()